In [1]:
print("Um cuspidor esta nascendo!")

Um cuspidor esta nascendo!


In [1]:
!pip install llama-index-readers-wikipedia -q

In [2]:
!pip install gradio duckdb duckdb-engine pandas unidecode --quiet


In [ ]:
!pip install llama-index llama-index-experimental llama-index-llms-groq -q

# Como Obter e Usar a API Key da Groq

## 1. Criar Conta ou Fazer Login
- Acesse [Groq Console](https://console.groq.com/login) e crie uma conta ou faça login se já possuir.

## 2. Navegar até a Seção de API Keys
- No painel da sua conta, vá até a página de [API Keys](https://console.groq.com/keys).

## 3. Criar uma Nova API Key
- Clique em **"Create API Key"**.
- Dê um nome descritivo à chave (por exemplo: `Colab Integration`) e confirme a criação.

## 4. Copiar a API Key
- Após a criação, copie a API key exibida.
- **Atenção:** Essa será a única vez que a chave será mostrada. Guarde-a em um local seguro.

## 5. Configurar a Chave no Colab
- **Boa prática:** Não coloque a chave diretamente no código. Em vez disso, use **Colab Secrets** ou variáveis de ambiente.

Exemplo usando variável de ambiente:

```python
import os

# Defina a variável de ambiente
os.environ["GROQ_API_KEY"] = "sua-api-key-aqui"

#ou ustilizando secret
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')


In [42]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

import duckdb
from llama_index.llms.groq import Groq
from sqlalchemy import create_engine, text



engine = create_engine("duckdb:///:memory:")
ducklake_db = "spitter_ducklake"
path = "/tmp/dklq_spitter_ducklake"

with engine.connect() as con:
    con.exec_driver_sql("INSTALL 'ducklake';")
    con.exec_driver_sql("LOAD 'ducklake';")
    con.exec_driver_sql(f"""ATTACH 'ducklake:metadata.ducklake' AS {ducklake_db} (DATA_PATH '{path}');""")
    con.exec_driver_sql(f"USE {ducklake_db}.bronze;")
    print("Databases anexados:")
    result = con.exec_driver_sql(f"""SHOW TABLES;""").fetchall()

    print(result)
    tables = [i[0] for i in result]
    print(tables)

    con.commit()

# con = duckdb.connect(database=':memory:')
# con.execute(f"""ATTACH 'ducklake:metadata.ducklake' AS {ducklake_db} (DATA_PATH '{path}');""")

# #con.execute("SHOW tables;").fetchall()
# con.execute(f"""SELECT
#     database_name,
#     schema_name,
#     table_name,
#     temporary
# FROM
#     duckdb_tables
# WHERE database_name= '{ducklake_db}';""").fetchall()




#con.execute(f"SHOW TABLES FROM spitter_ducklake.duckdb;").fetchall()
#con.execute("USE spitter_ducklake.bronze;")

from llama_index.core import SQLDatabase
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core import Settings

embed_model = Groq(model='llama-3.3-70b-versatile', api_key=os.environ.get("GROQ_API_KEY"))

sql_database = SQLDatabase(engine, include_tables=tables)
query_engine = NLSQLTableQueryEngine(sql_database,embed_model=embed_model)
#index = SQLStructStoreIndex(sql_database=db, include_tables=tables)
#with engine.connect() as con:
# print(con.execute(text(f"""SELECT * FROM spitter_ducklake.bronze.tb_bronze_escolas_gramame_geisel """)).fetchall())

Databases anexados:
[('tb_bronze_escolas_gramame_geisel',)]
['tb_bronze_escolas_gramame_geisel']


In [ ]:
from llama_index.core import Settings
from llama_index.core import PromptTemplate

Settings.llm = Groq(model='llama-3.3-70b-versatile', api_key=os.environ.get("GROQ_API_KEY"))

def get_table_statistcs(table, layer):
    ducklake_db = "spitter_ducklake"
    path = "/tmp/dklq_spitter_ducklake"
    with duckdb.connect(database=':memory:') as con:
      con.execute(f"""ATTACH 'ducklake:metadata.ducklake' AS {ducklake_db} (DATA_PATH '{path}');""")
      con.execute(f"USE {ducklake_db}.{layer};")
      desc = con.execute(f"""DESCRIBE FROM {table}""").fetchall()
      cols = ",".join([f'"{row[0]}"' for row in desc])
      ref_query = f"SELECT {cols} FROM {table}"

      desc_str = "\n".join([f"{col}: {dtype}" for col, dtype, *_ in desc])
      sample_df = con.execute(ref_query).fetchdf().head(10)
      sample_df = sample_df.apply(lambda col: col.astype(str).str.slice(0, 50) if col.dtype == "object" else col)
      sample_text = sample_df.to_string(index=False)


    return desc_str, sample_text, ref_query


def generate_sql(query, table, layer, last_query=False):
    llm = Settings.llm

    schema_table, sample_table,ref_query = get_table_statistcs(table, layer)
    ref_query = last_query if last_query else ref_query
    """"
        Gere a consulta com base nas colunas da tabela.\n
        Modifique apenas os campos solicitados; todas as demais partes da query devem permanecer iguais.\n
        Caso solicitado, voce devera remover campos da query e desfazer alteracões.\n
        A query final deve ser uma expressão SQL válida e executável em SQL ANSI.\n
        Retorne apenas a query, sem explicações, comentários ou texto adicional, sem aspas.\n

    """
    prompt = (
        "Você está trabalhando com uma tabela chamada {table} SQL presente no duckdb.\n"
        "Descrição:\n"
        "{schema_table}\n\n"
        "Algumas linhas da tabela\n"
        "{sample_table}\n\n"

        "Query atual:\n"
        "{last_query}\n\n"

        "Nova instrução:\n"
        "{query}\n\n"

        "Gere uma **nova query SQL** baseada na query atual, alterando apenas o necessário.\n"
        "A query final deve ser executável em SQL ANSI.\n"
        "Retorne apenas a query, sem aspas ou complemento.\n")


    sql_prompt = PromptTemplate(prompt).partial_format(
        schema_table=schema_table,
        table=table,
        sample_table=sample_table,
        query=query,
        last_query = last_query )

    sql_query = llm.predict(sql_prompt)
    #response = llm.complete(sql_query_response).text


    return sql_query




table = "tb_bronze_escolas_gramame_geisel"
layer="bronze"
query_gen = False
        #Query base:\n
        #{last_query}\n\n

In [122]:
query = "Realize a conversao na coluna escola para minusculo, e mantenha as demais como estao"
query_gen = generate_sql(query, table, layer, last_query=query_gen)

print("\n--- QUERY SQL GERADA ---")
print(query_gen)


--- QUERY SQL GERADA ---
SELECT UPPER(bairro) AS bairro, LOWER(escola) AS escola, fonte FROM tb_bronze_escolas_gramame_geisel


In [ ]:
from unidecode import unidecode
import traceback
import os

import re

class SpitterDucklakeOperator():
    def __init__(self,con,  path=None):

      self.ducklake_db = "spitter_ducklake"
      self.path_prefix = os.environ.get("TMP").replace("\\","/") if os.name == "nt" else "/tmp"
      self.path = path if path else f"{self.path_prefix}/dklq_{self.ducklake_db}"
      self.con = con
      self.init_ducklake()


    def init_ducklake(self):
      try:
        #con = duckdb.connect(database=':memory:')
        self.con.execute("INSTALL ducklake;")
        self.con.execute(f"DETACH DATABASE IF EXISTS {self.ducklake_db};")
        self.con.execute(f"ATTACH 'ducklake:metadata.ducklake' AS {self.ducklake_db} (DATA_PATH '{self.path}');")
        #return con
      except Exception as e:
        raise e

    def table_exist(self, table):
      try:
        self.con.execute(f"SELECT * FROM {table} LIMIT 0")
        return True
      except Exception as e:
        return False

    def normalize_name(self, col):
      col = unidecode(col).lower().strip()
      col = re.sub(r'[^a-z0-9_]', '_', col)
      col = re.sub(r'_+', '_', col)
      return col.strip('_').encode("utf-8", errors="ignore").decode("utf-8")




    def create_or_replace_table(self, source_table_name, output_table_name, layer, primary_key=False):
      try:
        self.con.execute(f"CREATE SCHEMA IF NOT EXISTS {self.ducklake_db}.{layer}")

        instruction_sql = ','.join(["{0} {1} {2}".format(
                                                          self.normalize_name(row[0]),
                                                          row[1],
                                                          "" if row[0] != primary_key else 'PRIMARY KEY'
                                                        ) for row in self.con.execute(f"DESCRIBE FROM {source_table_name}").fetchall()
                                  ]
                                    )

        self.con.execute(f"""CREATE OR REPLACE TABLE {self.ducklake_db}.{layer}.{output_table_name} ({instruction_sql})""")

        return True, "success"
      except Exception as e:
        return False, f"{e} -> {traceback.format_exc()}"

    def insert_table(self, source_table_name, output_table_name, layer,op_mode, primary_key=False):
      try:
        if not self.table_exist(f"{self.ducklake_db}.{layer}.{output_table_name}") or op_mode == "overwrite":
            self.create_or_replace_table(source_table_name, output_table_name, layer, primary_key)

        if primary_key:
          self.con.execute(f"INSERT OR REPLACE INTO {self.ducklake_db}.{layer}.{output_table_name} SELECT * FROM {source_table_name}")
        else:
          self.con.execute(f"INSERT INTO {self.ducklake_db}.{layer}.{output_table_name} SELECT * FROM {source_table_name}")

        return True, "success"
      except Exception as e:
        return False, f"{e} -> {traceback.format_exc()}"


In [ ]:
#csv operations
import duckdb
import pandas as pd
import traceback

class SpitterOperator():
    def __init__(self, file):
        self.df = pd.DataFrame()
        self.primary_key = ""
        self.cols = []
        self.con = duckdb.connect(database=':memory:')
        self.dklq = SpitterDucklakeOperator(self.con)
        self.table_name = ""
        self.op_mode = ""
        if file is not None:
            self.load_bronze(file)
        else:
            return "Envie um file CSV."

    def preview_bronze(self):
        norm_cols = ",".join([f'"{col}" AS {self.dklq.normalize_name(col)}' for col in self.cols])
        preview_df = self.con.execute(f"""
          SELECT {norm_cols} FROM tmp_{self.table_name} LIMIT 10
        """).fetchdf()
        return preview_df

    def set_primary_key(self, column_name):
        if column_name in self.df.columns:
            self.primary_key = column_name

    def set_table_op_mode(self, op_mode):
        self.op_mode = op_mode

    def persist_table_on_dklq(self, layer):
        result = self.dklq.insert_table(f"tmp_{self.table_name}", f"tb_{layer}_{self.table_name}", layer, self.op_mode, self.primary_key)
        if not result[0]:
          raise Exception(result[1])


    def load_bronze(self,file):

        try:

          self.table_name = self.dklq.normalize_name(file.name.replace("\\", "/").split("/")[-1].split(".")[0])
          self.con.execute(f"""
              CREATE TEMPORARY TABLE tmp_{self.table_name} AS
              SELECT * FROM read_csv_auto('{file.name}', header=True)
          """)
          self.cols = [row[0] for row in self.con.execute(f"DESCRIBE tmp_{self.table_name}").fetchall()]
        except Exception as e:
          raise (f"{e} -> {traceback.format_exc()}")


In [ ]:
import gradio as gr

def init_file_operator(file):
    spitter_operator = SpitterOperator(file)
    return spitter_operator, "bronze"

def preview_bronze(spitter_operator):
    return spitter_operator.preview_bronze()

def set_pk_column(spitter_operator, pk_column):
    if spitter_operator is not None:
        spitter_operator.set_primary_key(pk_column)
    return spitter_operator

#button options
def set_pk_chk_visibility(spitter_operator, checkbox_value):
    if checkbox_value:
        return gr.update(interactive=True, visible=True,choices=spitter_operator.cols)
    else:
        return gr.update(interactive=False, visible=False)

def set_op_chk_visibility(spitter_operator, layer):
    choices = ["append", "overwrite"] if layer == "bronze" else ["overwrite","upsert"]
    return gr.update(interactive=True, visible=True, choices= choices)


def set_table_op_mode_btn(spitter_operator, op_mode):
  spitter_operator.set_table_op_mode(op_mode)
  return f"Commit - table -> {spitter_operator.table_name} | op_mode -> {op_mode}", True

def publish_bronze_table(spitter_operator, commit_state, layer):
  if commit_state:
    spitter_operator.persist_table_on_dklq(layer)
    return "sucess"
  else:
    return "Error. Please check your settings!"




with gr.Blocks(title="# Arquitetura Medalhão - Data Spitter - CSV - multi Reads") as spitter_demo:
    gr.Markdown("# Arquitetura Medalhão (Bronze / Silver / Gold)")
    gr.Markdown("## Spitter Data Lakehouse Explorer")
    gr.Markdown("## Leia seus dados, configure as operacoes, e maos a obra. Apresentamos suporte a cargas incrementais e de sobrescrita")
    gr.Markdown("As tabelas criadas utilizarao o nome do arquivo como referencia")
    st_spitter_operator = gr.State(None)
    st_layer = gr.State("")

    #BronzeLayer
    with gr.Tab("Bronze Layer"):
        gr.Markdown("### Upload dos dados brutos")
        st_choices = gr.State([])
        st_commit_bronze = gr.State(False)

        csv_bronze = gr.File(label="Selecione um CSV",file_count="single", type="filepath")
        csv_bronze.upload(init_file_operator, inputs=csv_bronze, outputs=[st_spitter_operator,st_layer])


        bronze_btn = gr.Button("Preview Bronze")
        bronze_table = gr.DataFrame(label="Bronze Data")
        bronze_btn.click(preview_bronze, inputs=st_spitter_operator, outputs=bronze_table)

        gr.Markdown("### Limpeza dos dados brutos")


        op_dropdown = gr.Dropdown(label="Select the table operaration mode (required)", choices=[], interactive=False, visible=False)
        st_spitter_operator.change(set_op_chk_visibility,inputs=[st_spitter_operator, st_layer],outputs=op_dropdown)

        output_text = gr.Markdown("")
        op_dropdown.change(set_table_op_mode_btn, inputs=[st_spitter_operator, op_dropdown], outputs=[output_text, st_commit_bronze])

        alerta_md_component = gr.Markdown("")
        publish_bronze_btn = gr.Button(value=f"Commit", visible=True)
        publish_bronze_btn.click(publish_bronze_table, inputs=[st_spitter_operator, st_commit_bronze, st_layer], outputs=alerta_md_component)

    #SilverLayer
    with gr.Tab("Camada Silver"):

        pk_checkbox = gr.Checkbox(label="Does the table have a primary key?", value=False)
        pk_dropdown = gr.Dropdown(label="Select the primary key", choices=[], interactive=False, visible=False)
        pk_checkbox.change(set_pk_chk_visibility, inputs=[st_spitter_operator, pk_checkbox], outputs=pk_dropdown)
        pk_dropdown.change(set_pk_column, inputs=[st_spitter_operator, pk_dropdown], outputs=st_spitter_operator)



spitter_demo.launch(debug=True)
#spitter_demo.queue(show_error=True)
#spitter_demo.launch(share=True)



It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://28280389c75a1f11f4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://28280389c75a1f11f4.gradio.live
